In [1]:
#
# import utilitaires
#
import helpers as hlp
import helpers.split as split
import helpers.visual_words as vw
import helpers.features as feat

In [2]:
#
# parametres
#
configs = hlp.get_configs("config_overrides.json")

#
# train/test sets
# 
train, test = split.load(configs.split)
print("Train size", len(train))
print("Test size", len(test))
print()

#
# features orb
#
# mettre a jour train/test set features
configs.orb.read_only = False
orb_features = feat.load(configs.orb, None)
if configs.orb.force_generate or orb_features.train_features is None:
    feat.update_train_test(orb_features, train, test)

print("ORB features", orb_features.features.shape)
print("ORB train features", orb_features.train_features.shape)
print("ORB test features", orb_features.test_features.shape)
print()

#
# features sift
#
configs.sift.read_only = False
sift_features = feat.load(configs.sift, None)
if configs.sift.force_generate or sift_features.train_features is None:
    feat.update_train_test(sift_features, train, test)

print("SIFT features", sift_features.features.shape)
print("SIFT train features", sift_features.train_features.shape)
print("SIFT test features", sift_features.test_features.shape)

Train size 3711
Test size 928

ORB features (20906105, 32)
ORB train features (1435022, 32)
ORB test features (359717, 32)

SIFT features (14382049, 128)
SIFT train features (1167882, 128)
SIFT test features (293047, 128)


In [3]:
# ATTENTION: cette etape peut etre longue a la 1e execution
#            elle est sans feedback visuel, soyez patient
with hlp.Profile() as orb_bovw_profile:
    orb_bovw = vw.load_bovw(configs.orb_bovw, orb_features.train_features)

print("ORB BoVW loading:", orb_bovw_profile.round_duration(), "s")
print("ORB BoVW clusters:", orb_bovw.model.cluster_centers_.shape)

Construction Bag of Visual Words
ORB BoVW loading: 4.49 s
ORB BoVW clusters: (500, 32)


In [4]:
# ATTENTION: cette etape peut etre longue a la 1e execution
#            elle est sans feedback visuel, soyez patient
with hlp.Profile() as sift_bovw_profile:
    sift_bovw = vw.load_bovw(configs.sift_bovw, sift_features.train_features)

print("SIFT BoVW loading:", sift_bovw_profile.round_duration(), "s")
print("SIFT BoVW clusters:", sift_bovw.model.cluster_centers_.shape)

Construction Bag of Visual Words
SIFT BoVW loading: 5.87 s
SIFT BoVW clusters: (500, 128)
